In [49]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-06-04 13:36:44--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-06-04 13:36:44--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.57K  --.-KB/s    in 0s      

2021-06-04 13:36:44 (25.8 MB/s) - written to stdout [1608/1608]

setup Cola

In [50]:
import sparknlp
spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 3.0.3
Apache Spark version: 3.0.2


In [51]:
!pip install wget

In [5]:
import wget
wget.download('https://s3.amazonaws.com/data.patentsview.org/download/patent.tsv.zip','sample_data/patent.tsv.zip')

from zipfile import ZipFile
  
# specifying the zip file name
file_name = "sample_data/patent.tsv.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
patent.tsv                                     2021-03-04 23:42:36   6005805218
Extracting all the files now...
Done!


In [52]:
data = spark.read.csv('patent.tsv',
                      sep="\t",
                      header=True,
                      inferSchema=True)
data.printSchema()

root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- number: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- title: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- num_claims: integer (nullable = true)
 |-- filename: string (nullable = true)
 |-- withdrawn: integer (nullable = true)



In [7]:
data.show(5)

+--------+-------+--------+-------+----------+--------------------+--------------------+----+----------+-------------+---------+
|      id|   type|  number|country|      date|            abstract|               title|kind|num_claims|     filename|withdrawn|
+--------+-------+--------+-------+----------+--------------------+--------------------+----+----------+-------------+---------+
|10000000|utility|10000000|     US|2018-06-19|A frequency modul...|Coherent LADAR us...|  B2|        20|ipg180619.xml|        0|
|10000001|utility|10000001|     US|2018-06-19|The injection mol...|Injection molding...|  B2|        12|ipg180619.xml|        0|
|10000002|utility|10000002|     US|2018-06-19|The present inven...|Method for manufa...|  B2|         9|ipg180619.xml|        0|
|10000003|utility|10000003|     US|2018-06-19|The invention rel...|Method for produc...|  B2|        18|ipg180619.xml|        0|
|10000004|utility|10000004|     US|2018-06-19|The present inven...|Process of obtain...|  B2|    

In [53]:
data = data.filter(data['withdrawn']==0)
data = data.withColumn('year',data['date'].substr(1,4))

In [54]:
from pyspark.sql.types import IntegerType
data = data.withColumn('year',data['year'].cast(IntegerType()))

In [11]:
data.printSchema()

root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- number: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- title: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- num_claims: integer (nullable = true)
 |-- filename: string (nullable = true)
 |-- withdrawn: integer (nullable = true)
 |-- year: integer (nullable = true)



# **Prepocessing**

In [55]:
from sparknlp.base import DocumentAssembler

documentAssembler = DocumentAssembler() \
     .setInputCol('title') \
     .setOutputCol('document')

In [56]:
from sparknlp.annotator import Tokenizer

tokenizer = Tokenizer() \
     .setInputCols(['document']) \
     .setOutputCol('tokenized')

In [57]:
from sparknlp.annotator import Normalizer

normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') \
     .setLowercase(True)

In [58]:
from sparknlp.annotator import LemmatizerModel

lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemmatized')

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [59]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
from sparknlp.annotator import StopWordsCleaner

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('unigrams') \
     .setStopWords(eng_stopwords)

In [61]:
from sparknlp.annotator import NGramGenerator

ngrammer = NGramGenerator() \
    .setInputCols(['lemmatized']) \
    .setOutputCol('ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

In [62]:
from sparknlp.annotator import PerceptronModel

pos_tagger = PerceptronModel.pretrained('pos_anc') \
    .setInputCols(['document', 'lemmatized']) \
    .setOutputCol('pos')

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]


In [63]:
from sparknlp.base import Finisher

finisher = Finisher() \
     .setInputCols(['unigrams', 'ngrams', 'pos']) \

In [64]:
from pyspark.ml import Pipeline

pipeline = Pipeline() \
     .setStages([documentAssembler,                  
                 tokenizer,
                 normalizer,                  
                 lemmatizer,                  
                 stopwords_cleaner, 
                 pos_tagger,
                 ngrammer,  
                 finisher])

In [65]:
processed_review = pipeline.fit(data).transform(data)

In [66]:
processed_review.limit(5).show()

+--------+-------+--------+-------+----------+--------------------+--------------------+----+----------+-------------+---------+----+--------------------+--------------------+--------------------+
|      id|   type|  number|country|      date|            abstract|               title|kind|num_claims|     filename|withdrawn|year|   finished_unigrams|     finished_ngrams|        finished_pos|
+--------+-------+--------+-------+----------+--------------------+--------------------+----+----------+-------------+---------+----+--------------------+--------------------+--------------------+
|10000000|utility|10000000|     US|2018-06-19|A frequency modul...|Coherent LADAR us...|  B2|        20|ipg180619.xml|        0|2018|[coherent, ladar,...|[coherent, ladar,...|[JJ, NN, NN, NN, ...|
|10000001|utility|10000001|     US|2018-06-19|The injection mol...|Injection molding...|  B2|        12|ipg180619.xml|        0|2018|[injection, moldi...|[injection, moldi...|[NN, NN, NN, CC, ...|
|10000002|utili

In [67]:
from pyspark.sql import types as T
from pyspark.sql import functions as F

udf_join_arr = F.udf(lambda x: ' '.join(x), T.StringType())
processed_review  = processed_review.withColumn('finished_pos', udf_join_arr(F.col('finished_pos')))

In [68]:
pos_documentAssembler = DocumentAssembler() \
     .setInputCol('finished_pos') \
     .setOutputCol('pos_document')

In [69]:
pos_tokenizer = Tokenizer() \
     .setInputCols(['pos_document']) \
     .setOutputCol('pos')

In [70]:
pos_ngrammer = NGramGenerator() \
    .setInputCols(['pos']) \
    .setOutputCol('pos_ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

In [71]:
pos_finisher = Finisher() \
     .setInputCols(['pos', 'pos_ngrams']) \

In [72]:
pos_pipeline = Pipeline() \
     .setStages([pos_documentAssembler,                  
                 pos_tokenizer,
                 pos_ngrammer,  
                 pos_finisher])

In [73]:
processed_review = pos_pipeline.fit(processed_review).transform(processed_review)

In [74]:
processed_review.columns

['id',
 'type',
 'number',
 'country',
 'date',
 'abstract',
 'title',
 'kind',
 'num_claims',
 'filename',
 'withdrawn',
 'year',
 'finished_unigrams',
 'finished_ngrams',
 'finished_pos',
 'finished_pos_ngrams']

In [75]:
processed_review.show(5)

+--------+-------+--------+-------+----------+--------------------+--------------------+----+----------+-------------+---------+----+--------------------+--------------------+--------------------+--------------------+
|      id|   type|  number|country|      date|            abstract|               title|kind|num_claims|     filename|withdrawn|year|   finished_unigrams|     finished_ngrams|        finished_pos| finished_pos_ngrams|
+--------+-------+--------+-------+----------+--------------------+--------------------+----+----------+-------------+---------+----+--------------------+--------------------+--------------------+--------------------+
|10000000|utility|10000000|     US|2018-06-19|A frequency modul...|Coherent LADAR us...|  B2|        20|ipg180619.xml|        0|2018|[coherent, ladar,...|[coherent, ladar,...|[JJ, NN, NN, NN, ...|[JJ, NN, NN, NN, ...|
|10000001|utility|10000001|     US|2018-06-19|The injection mol...|Injection molding...|  B2|        12|ipg180619.xml|        0|

In [76]:
def filter_pos(words, pos_tags):
    return [word for word, pos in zip(words, pos_tags) 
            if pos in ['JJ', 'NN', 'NNS', 'VB', 'VBP']]

udf_filter_pos = F.udf(filter_pos, T.ArrayType(T.StringType()))

In [77]:
processed_review.columns

['id',
 'type',
 'number',
 'country',
 'date',
 'abstract',
 'title',
 'kind',
 'num_claims',
 'filename',
 'withdrawn',
 'year',
 'finished_unigrams',
 'finished_ngrams',
 'finished_pos',
 'finished_pos_ngrams']

In [99]:
processed_review.show(5)

+--------+-------+--------+-------+----------+--------------------+--------------------+----+----------+-------------+---------+----+--------------------+--------------------+--------------------+--------------------+
|      id|   type|  number|country|      date|            abstract|               title|kind|num_claims|     filename|withdrawn|year|   finished_unigrams|     finished_ngrams|        finished_pos| finished_pos_ngrams|
+--------+-------+--------+-------+----------+--------------------+--------------------+----+----------+-------------+---------+----+--------------------+--------------------+--------------------+--------------------+
|10000000|utility|10000000|     US|2018-06-19|A frequency modul...|Coherent LADAR us...|  B2|        20|ipg180619.xml|        0|2018|[coherent, ladar,...|[coherent, ladar,...|[JJ, NN, NN, NN, ...|[JJ, NN, NN, NN, ...|
|10000001|utility|10000001|     US|2018-06-19|The injection mol...|Injection molding...|  B2|        12|ipg180619.xml|        0|

# **Vectorization**

In [78]:
from pyspark.ml.feature import CountVectorizer

tfizer = CountVectorizer(inputCol='finished_ngrams', outputCol='tf_features')
tf_model = tfizer.fit(processed_review)
tf_result = tf_model.transform(processed_review)

In [80]:
from pyspark.ml.feature import IDF

idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

# **LDA**

In [81]:
from pyspark.ml.clustering import LDA

num_topics = 6
max_iter = 10

lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features')
lda_model = lda.fit(tfidf_result)

In [82]:
vocab = tf_model.vocabulary

def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
       
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

In [83]:
num_top_words = 20

topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=90)

+-----+------------------------------------------------------------------------------------+
|topic|                                                                          topicWords|
+-----+------------------------------------------------------------------------------------+
|    0|                    [with, for, a, and, of, use, control, assembly, method, produce]|
|    1|                [a, for, and, method, apparatus, device, system, of, in, method_and]|
|    2|                   [of, and, method, for, device, method_of, the, a, apparatus, use]|
|    3|                  [and, a, method, for, of, device, display, apparatus, system, the]|
|    4|[and, method, for, system, of, a, and_method, method_and, communication, method_for]|
|    5| [process, of, for, method, and, method_for, and_method, device, the, semiconductor]|
+-----+------------------------------------------------------------------------------------+

